# Project 2: Ames Housing Data

## Data Cleaning

### Importing the Data

In [1]:
import pandas as pd
import numpy as np

First I'll bring in the raw dataset.

In [2]:
ames_raw = '../datasets/train.csv'
df = pd.read_csv(ames_raw)
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


I want to take a closer look at the `Id` and `PID` features.

In [3]:
df.Id.nunique()

2051

In [4]:
df.shape

(2051, 81)

`Id` and `PID` appear to be functionally redundant as unique identifiers, so I'll drop `PID` and set the dataframe to index on `Id`.

In [5]:
df.set_index('Id', inplace=True)
df.drop(columns='PID', inplace=True)

Next I'll take a look to see how many null values need to be addressed.

In [6]:
df.isnull().sum().sort_values(ascending=False).head(27)

Pool QC           2042
Misc Feature      1986
Alley             1911
Fence             1651
Fireplace Qu      1000
Lot Frontage       330
Garage Qual        114
Garage Cond        114
Garage Yr Blt      114
Garage Finish      114
Garage Type        113
Bsmt Exposure       58
BsmtFin Type 2      56
Bsmt Qual           55
BsmtFin Type 1      55
Bsmt Cond           55
Mas Vnr Type        22
Mas Vnr Area        22
Bsmt Half Bath       2
Bsmt Full Bath       2
Bsmt Unf SF          1
Total Bsmt SF        1
BsmtFin SF 1         1
BsmtFin SF 2         1
Garage Cars          1
Garage Area          1
Exterior 2nd         0
dtype: int64

### Dealing with the Worst Offenders

`Pool QC` is mostly null, so I'll take a look at the values it does have.

In [7]:
df['Pool QC'].value_counts()

Gd    4
TA    2
Fa    2
Ex    1
Name: Pool QC, dtype: int64

`Pool QC` doesn't contain any values for a missing pool, so I will fill those nulls as 'NA' to signify that these are properties that do not have pools.

In [8]:
df['Pool QC'].fillna('NA', inplace=True)

Now that the nulls in `Pool QC` have been filled, I'll take a look at the values in `Misc Feature` next.

In [9]:
df['Misc Feature'].value_counts()

Shed    56
Gar2     4
Othr     3
TenC     1
Elev     1
Name: Misc Feature, dtype: int64

Since these are uncommon features in a home, I think it's reasonable to again impute a value of 'NA' for the nulls here.

In [10]:
df['Misc Feature'].fillna('NA', inplace=True)

`Fence` and `Alley` are two remaining features with mostly null values. The categorical values for `Fence` in the provided data dictionary aren't entirely clear so I'm not confident about their consistent ability to influence price. However, the mere existence of a fence could be relevant and it's easy to classify, so I'll convert this column to a binary one where a value of 1 indicates that a property has a fence and a 0 indicates that the property has no fence.

In [11]:
fence_quality = ['MnPrv', 'GdPrv', 'GdWo', 'MnWw']
for quality in fence_quality:
    df.Fence = df.Fence.str.replace(quality, '1')
df.Fence.fillna(0, inplace=True)
df.Fence = df.Fence.apply(lambda x: int(x))
df.Fence.head(8)

Id
109     0
544     0
153     0
318     0
255     0
138     0
2827    0
145     1
Name: Fence, dtype: int64

The `Fence` column is now numeric and indicates whether or not the property has a fence. I'll run a similar process on `Alley` to convert that feature into a binary one rather than a categorical one. All properties with an alley, regardless of material, will have an `Alley` value of 1, and all other properties will have their null value filled with a 0.

In [12]:
alley_quality = ['Grvl', 'Pave']
for quality in alley_quality:
    df.Alley = df.Alley.str.replace(quality, '1')
df.Alley.fillna(0, inplace=True)
df.Alley = df.Alley.apply(lambda x: int(x))
df.Alley.head(8)

Id
109     0
544     0
153     0
318     0
255     0
138     0
2827    0
145     0
Name: Alley, dtype: int64

Another feature with plenty of null values is `Fireplace Qu`. In my opinion, the categorical descriptions for `Fireplace Qu` in the provided data dictionary lack precision. Since we already have a numeric feature (`Fireplaces`) indicating how many fireplaces are in each property, I feel comfortable dropping `Fireplace Qu` from the dataset.

In [13]:
df.drop(columns='Fireplace Qu', inplace=True)

### Garage Nulls

Next I'll focus on the garage-related features. First I'll create a function that can quickly display which properties have null values in a particular column.

In [14]:
def has_null(col):
    null_mask = (df[col].isnull())
    print(df[null_mask])
    return

has_null('Garage Cars')

      MS SubClass MS Zoning  Lot Frontage  Lot Area Street  Alley Lot Shape  \
Id                                                                            
2237           70        RM          50.0      9060   Pave      0       Reg   

     Land Contour Utilities Lot Config  ... Screen Porch Pool Area Pool QC  \
Id                                      ...                                  
2237          Lvl    AllPub     Inside  ...            0         0      NA   

     Fence Misc Feature Misc Val  Mo Sold  Yr Sold  Sale Type  SalePrice  
Id                                                                        
2237     1           NA        0        3     2007        WD      150909  

[1 rows x 78 columns]


I'll take a closer look at property 2237.

In [15]:
df.loc[2237, df.columns.str.contains('Garage')]

Garage Type      Detchd
Garage Yr Blt       NaN
Garage Finish       NaN
Garage Cars         NaN
Garage Area         NaN
Garage Qual         NaN
Garage Cond         NaN
Name: 2237, dtype: object

This is the only property that is missing values for `Garage Cars` and `Garage Area`, and its `Garage Type` value indicates that it does have a garage. Since it's only one property and it has no other garage information from which to infer, I'm going to drop this row.

In [16]:
df.drop(index=2237, inplace=True)

Next I want to see if the same properties are missing all the various garage values.

In [17]:
df[df['Garage Type'].isnull() &
   df['Garage Yr Blt'].isnull() &
   df['Garage Finish'].isnull() &
   df['Garage Qual'].isnull() &
   df['Garage Cond'].isnull()]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
2243,190,C (all),75.0,8250,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,6,2007,WD,119600
330,160,RM,21.0,1476,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,3,2010,WD,76000
2278,20,RL,70.0,8402,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,12,2007,New,147000
2235,50,RM,50.0,7288,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,9,2007,WD,129850
2084,30,RL,58.0,9098,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,7,2007,WD,86000
728,30,C (all),72.0,9392,Pave,0,Reg,Lvl,AllPub,Corner,...,0,0,NA,0,NA,0,9,2009,WD,44000
781,20,RL,NaN,9000,Pave,0,Reg,Lvl,AllPub,Corner,...,0,0,NA,0,NA,0,6,2009,WD,63900
1492,30,RL,63.0,13907,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,7,2008,WD,108000
1514,30,RL,67.0,5604,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,4,2008,WD,98000


Since the same 113 properties are missing garage-related values, we can infer that these properties do not have garages. I will fill those values as 'NA' so they will be appropriately categorized when I one-hot encode the garage features. However, I first want to check the years that garages were built because values of 0 in that numeric column will distort the rest of the column's values. I will compare the years that the garages were built with the years the properties were built to see how many garages were built after the original property construction.

In [18]:
def compare_garage_year(data):
    counter = 0
    for row in data.index:
        if df.loc[row]['Garage Yr Blt'] - df.loc[row]['Year Built'] > 0:
            counter += 1
    return counter

compare_garage_year(df)

362

According to my `compare_garage_year` function, 362 properties have garages with different build years than the property itself. That number is higher than I would like, but since we already have so many other features for the garages, 362 is still low enough that I feel comfortable dropping the `Garage Yr Blt` column. I will also fill the null values of the four remaining garage features, then check the cleaning progress.

In [19]:
df.drop(columns = 'Garage Yr Blt', inplace=True)

df['Garage Type'].fillna('NA', inplace=True)
df['Garage Finish'].fillna('NA', inplace=True)
df['Garage Qual'].fillna('NA', inplace=True)
df['Garage Cond'].fillna('NA', inplace=True)

Now that the garages are cleaned, I'll deal with the basement nulls.

### Basement Nulls

First I want to see how many basement nulls I need to address.

In [20]:
basement_cols = df.columns[df.columns.str.contains('Bsmt')]

df[basement_cols].isnull().sum()

Bsmt Qual         55
Bsmt Cond         55
Bsmt Exposure     58
BsmtFin Type 1    55
BsmtFin SF 1       1
BsmtFin Type 2    56
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
dtype: int64

Next, I want to find the one property that is null for `BsmtFin Type 1`.

In [21]:
has_null('BsmtFin SF 1')

      MS SubClass MS Zoning  Lot Frontage  Lot Area Street  Alley Lot Shape  \
Id                                                                            
1342           20        RM          99.0      5940   Pave      0       IR1   

     Land Contour Utilities Lot Config  ... Screen Porch Pool Area Pool QC  \
Id                                      ...                                  
1342          Lvl    AllPub        FR3  ...            0         0      NA   

     Fence Misc Feature Misc Val  Mo Sold  Yr Sold  Sale Type  SalePrice  
Id                                                                        
1342     1           NA        0        4     2008      ConLD      79000  

[1 rows x 77 columns]


I'll take a closer look to see if it has other basement nulls.

In [22]:
df.loc[1342, basement_cols]

Bsmt Qual         NaN
Bsmt Cond         NaN
Bsmt Exposure     NaN
BsmtFin Type 1    NaN
BsmtFin SF 1      NaN
BsmtFin Type 2    NaN
BsmtFin SF 2      NaN
Bsmt Unf SF       NaN
Total Bsmt SF     NaN
Bsmt Full Bath    NaN
Bsmt Half Bath    NaN
Name: 1342, dtype: object

This property doesn't seem to have a basement at all, so I will fill its numerical basement values with 0 and its categorical basement values with NA.

In [23]:
df.loc[1342, 'Total Bsmt SF'] = 0
df.loc[1342, 'Bsmt Full Bath'] = 0
df.loc[1342, 'Bsmt Half Bath'] = 0
df.loc[1342, 'Bsmt Unf SF'] = 0
df.loc[1342, 'BsmtFin SF 2'] = 0
df.loc[1342, 'BsmtFin SF 1'] = 0
df.loc[1342, 'Bsmt Qual'] = 'NA'
df.loc[1342, 'Bsmt Cond'] = 'NA'
df.loc[1342, 'Bsmt Exposure'] = 'NA'
df.loc[1342, 'BsmtFin Type 1'] = 'NA'
df.loc[1342, 'BsmtFin Type 2'] = 'NA'

There should be one more property with a null value for `Bsmt Full Bath`.

In [24]:
has_null('Bsmt Full Bath')

      MS SubClass MS Zoning  Lot Frontage  Lot Area Street  Alley Lot Shape  \
Id                                                                            
1498           20        RL         123.0     47007   Pave      0       IR1   

     Land Contour Utilities Lot Config  ... Screen Porch Pool Area Pool QC  \
Id                                      ...                                  
1498          Lvl    AllPub     Inside  ...            0         0      NA   

     Fence Misc Feature Misc Val  Mo Sold  Yr Sold  Sale Type  SalePrice  
Id                                                                        
1498     0           NA        0        7     2008        WD      284700  

[1 rows x 77 columns]


I'll take a closer look at 1498.

In [25]:
df.loc[1498, basement_cols]

Bsmt Qual         NaN
Bsmt Cond         NaN
Bsmt Exposure     NaN
BsmtFin Type 1    NaN
BsmtFin SF 1        0
BsmtFin Type 2    NaN
BsmtFin SF 2        0
Bsmt Unf SF         0
Total Bsmt SF       0
Bsmt Full Bath    NaN
Bsmt Half Bath    NaN
Name: 1498, dtype: object

Since it's the only property left in the dataset with missing basement bathroom values, I will manually set those values to 0, but then I will start programatically filling in the rest of the dataset's basement nulls.

In [26]:
df.loc[1498, 'Bsmt Full Bath'] = 0
df.loc[1498, 'Bsmt Half Bath'] = 0

I want to get a quick look at whether all the properties with those null basement values have any existing values that could be overwritten when I pass the properties through a function.

In [27]:
no_basement = (df['Total Bsmt SF'] == 0)

df[no_basement][basement_cols]

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath
Id,,,,,,,,,,,
807,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
811,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
781,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
888,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1554,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2740,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2242,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2745,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
84,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0


Since there don't seem to be any stray values for these features, I'll write a function to fill all the null categories and pass the dataframe through it.

In [28]:
def fill_basement_nulls(data):
    for row in data.index:
        if data.loc[row, 'Total Bsmt SF'] == 0:
            data.loc[row, 'Bsmt Qual'] = 'NA'
            data.loc[row, 'Bsmt Cond'] = 'NA'
            data.loc[row, 'Bsmt Exposure'] = 'NA'
            data.loc[row, 'BsmtFin Type 1'] = 'NA'
            data.loc[row, 'BsmtFin Type 2'] = 'NA'
    return

fill_basement_nulls(df)

In [29]:
df[basement_cols].isnull().sum()

Bsmt Qual         0
Bsmt Cond         0
Bsmt Exposure     3
BsmtFin Type 1    0
BsmtFin SF 1      0
BsmtFin Type 2    1
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
dtype: int64

To address these last four basement nulls, I'll first figure out which properties have `Bsmt Exposure` nulls.

In [30]:
has_null('Bsmt Exposure')

      MS SubClass MS Zoning  Lot Frontage  Lot Area Street  Alley Lot Shape  \
Id                                                                            
1797           60        FV          81.0     10411   Pave      0       Reg   
67             20        RL          73.0      8987   Pave      0       Reg   
2780           60        RL          65.0     14006   Pave      0       IR1   

     Land Contour Utilities Lot Config  ... Screen Porch Pool Area Pool QC  \
Id                                      ...                                  
1797          Lvl    AllPub     Corner  ...            0         0      NA   
67            Lvl    AllPub     Inside  ...            0         0      NA   
2780          Lvl    AllPub     Inside  ...            0         0      NA   

     Fence Misc Feature Misc Val  Mo Sold  Yr Sold  Sale Type  SalePrice  
Id                                                                        
1797     0           NA        0        7     2007        New  

I have no reasonable way of inferring the `Bsmt Exposure` for those three properties, and because it's only three properties I believe I can drop them from the training dataset without any adverse effects on the model.

In [31]:
df.drop(index=1797, inplace=True)
df.drop(index=67, inplace=True)
df.drop(index=2780, inplace=True)

Now I'll look for the final basement null.

In [32]:
has_null('BsmtFin Type 2')

     MS SubClass MS Zoning  Lot Frontage  Lot Area Street  Alley Lot Shape  \
Id                                                                           
445           20        RL          85.0     10655   Pave      0       IR1   

    Land Contour Utilities Lot Config  ... Screen Porch Pool Area Pool QC  \
Id                                     ...                                  
445          Lvl    AllPub     Inside  ...            0         0      NA   

    Fence Misc Feature Misc Val  Mo Sold  Yr Sold  Sale Type  SalePrice  
Id                                                                       
445     0           NA        0       10     2009        WD      284000  

[1 rows x 77 columns]


I want to take a closer look at this property.

In [33]:
df.loc[445, basement_cols]

Bsmt Qual           Gd
Bsmt Cond           TA
Bsmt Exposure       No
BsmtFin Type 1     GLQ
BsmtFin SF 1      1124
BsmtFin Type 2     NaN
BsmtFin SF 2       479
Bsmt Unf SF       1603
Total Bsmt SF     3206
Bsmt Full Bath       1
Bsmt Half Bath       0
Name: 445, dtype: object

This property has a large basement and half of it is finished, with a substantial portion rated as good living quarters. We can assume that the `BsmtFin Type 2` is not also good living quarters, otherwise it would be grouped in with `BsmtFin Type 1`. Since we can see that the space is 479 square feet, and we can reasonably infer that the large living quarters contains the basement's only bathroom, we can also infer that the space is a rec room rather than another living quarters. Therefore, I will impute the value for an average rec room.

In [34]:
df.loc[445, 'BsmtFin Type 2'] = 'Rec'

And with that, I'm done cleaning the basement null values.

### Last Miscellaneous Nulls

In [35]:
df.isnull().sum().sort_values(ascending=False).head()

Lot Frontage    330
Mas Vnr Type     22
Mas Vnr Area     22
SalePrice         0
Foundation        0
dtype: int64

First I'll verify that the same 22 properties are missing both `Mas Vnr Area` and `Mas Vnr Area`.

In [36]:
df[df['Mas Vnr Area'].isnull() &
   df['Mas Vnr Type'].isnull()]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
2393,60,RL,103.0,12867,Pave,0,IR1,Lvl,AllPub,Corner,...,0,0,NA,0,NA,0,7,2006,New,344133
2383,20,RL,107.0,13891,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,9,2006,New,465000
539,20,RL,67.0,10083,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,8,2009,WD,184900
518,20,FV,90.0,7993,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,10,2009,New,225000
2824,20,RL,75.0,8050,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,4,2006,WD,117250
1800,60,FV,112.0,12217,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,12,2007,New,310013
1455,60,RL,75.0,9473,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,3,2008,WD,237000
1120,20,RL,87.0,10037,Pave,0,Reg,Lvl,AllPub,Corner,...,0,0,NA,0,NA,0,8,2008,WD,247000
1841,120,FV,35.0,4274,Pave,1,IR1,Lvl,AllPub,Inside,...,0,0,NA,0,NA,0,11,2007,New,199900


Since it is the same 22 properties, I'll take a look at the rest of the `Mas Vnr Type` feature.

In [37]:
df['Mas Vnr Type'].value_counts()

None       1216
BrkFace     628
Stone       168
BrkCmn       13
Name: Mas Vnr Type, dtype: int64

Since most properties have no masonry work, I'll impute the mode of 'None' and 0 for `Mas Vnr Type` and `Mas Vnr Area`.

In [38]:
df['Mas Vnr Area'].fillna(0, inplace=True)
df['Mas Vnr Type'].fillna('None', inplace=True)

df.isnull().sum().sort_values(ascending=False).head()

Lot Frontage    330
SalePrice         0
Foundation        0
Roof Matl         0
Exterior 1st      0
dtype: int64

We have too many nulls for `Lot Frontage` to drop those properties from our dataset, but not enough nulls to drop the feature entirely. Since it is unlikely that a property truly has zero linear feet of `Lot Frontage` I will impute the mean value.

In [39]:
df['Lot Frontage'].fillna(value=df['Lot Frontage'].mean(), inplace=True)

Now that I've cleaned `Lot Frontage` I'll verify that there are no more remaining null values in the dataset.

In [40]:
df.isnull().sum().sum()

0

The data is clean so I'll export it as a new CSV file designated `train_clean.csv`.

In [41]:
# df.to_csv('../datasets/train_clean.csv')

Now I'll move on to EDA and modeling in the next notebook.